This notebook is a repeat of the 01-example-patient-heart-failure.ipynb

https://github.com/MIT-LCP/mimic-workshop/blob/master/intro_to_mimic/01-example-patient-heart-failure.ipynb

In [2]:
import os
import csv
import time
from io import StringIO

import psycopg
from sqlalchemy import create_engine

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
print(f'DBHOST: {os.environ["DBHOST"]}')
print(f'DBUSER: {os.environ["DBUSER"]}')
print(f'DBPORT: {os.environ["DBPORT"]}')
print(f'DBNAME: {os.environ["DBNAME"]}')


DBHOST: irl.warthog-trout.ts.net
DBUSER: jrizzo
DBPORT: 5432
DBNAME: mimic


In [6]:
URI = f'postgresql+psycopg://{os.environ["DBUSER"]}:{os.environ["DBPASS"]}@{os.environ["DBHOST"]}:{os.environ["DBPORT"]}/{os.environ["DBNAME"]}'
QUERY = '''
    select ce.*, ie.*, di.*, ad.*, p.*
    from mimiciii.chartevents ce
	    inner join mimiciii.d_items di
		    ON ce.itemid = di.itemid
	    inner join mimiciii.icustays ie
		    ON ce.icustay_id = ie.icustay_id
	    inner join mimiciii.admissions ad
		    on ce.hadm_id = ad.hadm_id
	    inner join mimiciii.patients p 
		    on ce.subject_id = p.subject_id
    where ce.subject_id in (
	    select a.subject_id
	    from mimiciii.admissions a
	    where a.diagnosis like '%%TRANSPLANT%%'
	    order by a.subject_id asc
    )
    order by ce.charttime
'''

In [7]:
engine = create_engine(URI)
with engine.connect() as connection:
    ce = pd.read_sql_query(QUERY, con=connection, parse_dates=['charttime', 'intime'])

In [8]:
ce.keys()

Index(['row_id', 'subject_id', 'hadm_id', 'icustay_id', 'itemid', 'charttime',
       'storetime', 'cgid', 'value', 'valuenum', 'valueuom', 'warning',
       'error', 'resultstatus', 'stopped', 'row_id', 'subject_id', 'hadm_id',
       'icustay_id', 'dbsource', 'first_careunit', 'last_careunit',
       'first_wardid', 'last_wardid', 'intime', 'outtime', 'los', 'row_id',
       'itemid', 'label', 'abbreviation', 'dbsource', 'linksto', 'category',
       'unitname', 'param_type', 'conceptid', 'row_id', 'subject_id',
       'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime',
       'diagnosis', 'hospital_expire_flag', 'has_chartevents_data', 'row_id',
       'subject_id', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag'],
      dtype='object')

In [9]:
ce["hours"] = (ce["charttime"] - ce["intime"]).dt.total_seconds() / 3600

In [10]:
# Export the data to a CSV file
ce.to_csv('ce.csv', index=False)

Did the patient's vital signs breach any alarm thresholds?

In [ ]:
# Set x equal to the times
x_hr = ce.hours[ce.label=='Heart Rate']

# Set y equal to the heart rates
y_hr = ce.valuenum[ce.label=='Heart Rate']

# Plot time against heart rate
plt.figure(figsize=(14, 6))
plt.plot(x_hr,y_hr)


plt.xlabel('Time',fontsize=16)
plt.ylabel('Heart rate',fontsize=16)
plt.title('Heart rate over time from admission to the intensive care unit')

In [ ]:
plt.figure(figsize=(14, 6))

plt.plot(ce.hours[ce.label=='Respiratory Rate'], 
         ce.valuenum[ce.label=='Respiratory Rate'],
         'k+', markersize=10, linewidth=4)

plt.plot(ce.hours[ce.label=='Resp Alarm - High'], 
         ce.valuenum[ce.label=='Resp Alarm - High'],
         'm--')

plt.plot(ce.hours[ce.label=='Resp Alarm - Low'], 
         ce.valuenum[ce.label=='Resp Alarm - Low'],
         'm--')

plt.xlabel('Time',fontsize=16)
plt.ylabel('Respiratory rate',fontsize=16)
plt.title('Respiratory rate over time from admission, with upper and lower alarm thresholds')
plt.ylim(0,55)

What is patient's level of consciousness?

In [ ]:
# Display the first few rows of the GCS eye response data
ce[ce.label=='GCS - Eye Opening'].head()

In [ ]:
# Prepare the size of the figure
plt.figure(figsize=(18, 10))

# Set x equal to the times
x_hr = ce.hours[ce.label=='Heart Rate']

# Set y equal to the heart rates
y_hr = ce.valuenum[ce.label=='Heart Rate']


plt.plot(x_hr,y_hr)

plt.plot(ce.hours[ce.label=='Respiratory Rate'], 
         ce.valuenum[ce.label=='Respiratory Rate'],
         'k', markersize=6)

# Add a text label to the y-axis
plt.text(-20,155,'GCS - Eye Opening',fontsize=14)
plt.text(-20,150,'GCS - Motor Response',fontsize=14)
plt.text(-20,145,'GCS - Verbal Response',fontsize=14)   

# Iterate over list of GCS labels, plotting around 1 in 10 to avoid overlap
for i, txt in enumerate(ce.value[ce.label=='GCS - Eye Opening'].values):
    if np.mod(i,6)==0 and i < 65:
        plt.annotate(txt, (ce.hours[ce.label=='GCS - Eye Opening'].values[i],155),fontsize=14)
        
for i, txt in enumerate(ce.value[ce.label=='GCS - Motor Response'].values):
    if np.mod(i,6)==0 and i < 65:
        plt.annotate(txt, (ce.hours[ce.label=='GCS - Motor Response'].values[i],150),fontsize=14)

for i, txt in enumerate(ce.value[ce.label=='GCS - Verbal Response'].values):
    if np.mod(i,6)==0 and i < 65:
        plt.annotate(txt, (ce.hours[ce.label=='GCS - Verbal Response'].values[i],145),fontsize=14)

plt.title('Vital signs and Glasgow Coma Scale over time from admission',fontsize=16)

plt.xlabel('Time (hours)',fontsize=16)
plt.ylabel('Heart rate or GCS',fontsize=16)
plt.ylim(10,165)

In [13]:
# load outputs from the patient
query = """
select de.icustay_id
    -- , (strftime('%s',de.charttime)-strftime('%s',ie.intime))/60.0/60.0 as HOURS
    , de.charttime
    , ie.intime
    , di.label
    , de.value
    , de.valueuom
from mimiciii.outputevents de 
inner join mimiciii.icustays ie
    on de.icustay_id = ie.icustay_id
inner join mimiciii.d_items di
    on de.itemid = di.itemid
where de.subject_id = 40080
order by charttime;
"""

with engine.connect() as connection:
    oe = pd.read_sql_query(query, connection)

oe["hours"] = (oe["charttime"] - oe["intime"]).dt.total_seconds() / 3600

In [ ]:
oe.head()

In [ ]:
plt.figure(figsize=(14, 10))

plt.figure(figsize=(14, 6))
plt.title('Fluid output over time')

plt.plot(oe.hours, 
         oe.value.cumsum()/1000, 
         'ro', markersize=8, label='Output volume, L')

plt.xlim(0,72)
plt.ylim(0,10)
plt.legend()

In [16]:
query = """
select de.icustay_id
  -- , (strftime('%s',de.starttime)-strftime('%s',ie.intime))/60.0/60.0 as HOURS_START
  -- , (strftime('%s',de.endtime)-strftime('%s',ie.intime))/60.0/60.0 as HOURS_END
  , de.starttime
  , ie.intime
  , de.endtime
  , de.linkorderid
  , di.label
  , de.amount
  , de.amountuom
  , de.rate
  , de.rateuom
from mimiciii.inputevents_mv de 
inner join mimiciii.icustays ie
  on de.icustay_id = ie.icustay_id
inner join mimiciii.d_items di
  on de.itemid = di.itemid
where de.subject_id = 40080
order by endtime;
"""

with engine.connect() as connection:
    ie = pd.read_sql_query(query, connection)

ie["hours_start"] = (ie["starttime"] - ie["intime"]).dt.total_seconds() / 3600
ie["hours_end"] = (ie["endtime"] - ie["intime"]).dt.total_seconds() / 3600

In [ ]:
ie.head()

In [ ]:
ie['label'].unique()

In [ ]:
plt.figure(figsize=(14, 10))

# Plot the cumulative input against the cumulative output
plt.plot(ie.hours_end[ie.amountuom=='mL'], 
         ie.amount[ie.amountuom=='mL'].cumsum()/1000, 
         'go', markersize=8, label='Intake volume, L')

plt.plot(oe.hours, 
         oe.value.cumsum()/1000, 
         'ro', markersize=8, label='Output volume, L')

plt.title('Fluid balance over time',fontsize=16)
plt.xlabel('Hours',fontsize=16)
plt.ylabel('Volume (litres)',fontsize=16)
# plt.ylim(0,38)
plt.legend()